In [12]:
import pandas as pd
import seaborn as sns
import sys
import string
print('Original sys.path:', sys.path)

# Append a new directory to sys.path
sys.path.append('C:/Users/joepa/OneDrive/Desktop/computerScienceYear4/Data Mining/summary_eval/summary_eval')

# Print the updated sys.path
print('Updated sys.path:', sys.path)
from summary_eval.data import summary_df, prompts_df
from summary_eval.settings import TRAIN_SIZE
from summary_eval.testing import cross_validate

merged_df = pd.merge(summary_df, prompts_df, on='prompt_id')

Original sys.path: ['c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\notebooks', 'C:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval', 'C:\\Users\\joepa\\anaconda3\\python39.zip', 'C:\\Users\\joepa\\anaconda3\\DLLs', 'C:\\Users\\joepa\\anaconda3\\lib', 'C:\\Users\\joepa\\anaconda3', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv', '', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\win32', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\Pythonwin', 'C:/Users/joepa/OneDrive/Desktop/computerScienceYear4/Data M

In [13]:
#PREPROCESSING
from typing import List
import re
from autocorrect import Speller
import nltk
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

def pre_process_whitespace(text: str) -> str:
    # Replace all whitespace with a single space
    text = ' '.join(text.split())
    
    # Remove leading and trailing whitespace
    text = text.strip()
    return text

merged_df["text"] = merged_df["text"].apply(lambda x: pre_process_whitespace(x))

nltk.download('punkt')

def correct_spelling(text: str) -> str:
    spell = Speller(lang='en')
    corrected_text = []
    for word in nltk.word_tokenize(text):
        corrected_text.append(spell(word))

    return ' '.join(corrected_text)

# Apply correct_spelling function to 'text' column in merged_df
merged_df['text'] = merged_df['text'].apply(correct_spelling)

print(merged_df)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joepa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


        student_id prompt_id  \
0     000e8c3c7ddb    814d6b   
1     0020ae56ffbf    ebad26   
2     004e978e639e    3b9047   
3     005ab0199905    3b9047   
4     0070c9e7af47    814d6b   
...            ...       ...   
7160  ff7c7e70df07    ebad26   
7161  ffc34d056498    3b9047   
7162  ffd1576d2e1b    3b9047   
7163  ffe4a98093b2    39c16e   
7164  fffbccfd8a08    ebad26   

                                                   text   content   wording  \
0     The third wave was an experimental see how peo...  0.205683  0.380538   
1     They would rub it up with soda to make the sme... -0.548304  0.506755   
2     In Egypt , there were many occupations and soc...  3.128928  4.231226   
3     The highest class was Pharaoh these people wer... -0.210614 -0.471415   
4     The Third Wave developed rapidly because the s...  3.272894  3.219757   
...                                                 ...       ...       ...   
7160  They used all sorts of chemical connections to...  0.205

In [14]:
def count_specific_punctuation(text):
    punctuation_counts = {
        'num_commas': text.count(','),
        'num_quotation_marks': text.count('"'),
        'num_brackets': text.count('(') + text.count(')'),
        'num_full_stops': text.count('.')
    }
    return pd.Series(punctuation_counts)

# Apply the function to each text in the DataFrame
summary_df = pd.concat([summary_df, summary_df['text'].apply(count_specific_punctuation)], axis=1)

# Display the DataFrame
print(summary_df)

        student_id prompt_id  \
0     000e8c3c7ddb    814d6b   
1     0020ae56ffbf    ebad26   
2     004e978e639e    3b9047   
3     005ab0199905    3b9047   
4     0070c9e7af47    814d6b   
...            ...       ...   
7160  ff7c7e70df07    ebad26   
7161  ffc34d056498    3b9047   
7162  ffd1576d2e1b    3b9047   
7163  ffe4a98093b2    39c16e   
7164  fffbccfd8a08    ebad26   

                                                   text   content   wording  \
0     The third wave was an experimentto see how peo...  0.205683  0.380538   
1     They would rub it up with soda to make the sme... -0.548304  0.506755   
2     In Egypt, there were many occupations and soci...  3.128928  4.231226   
3     The highest class was Pharaohs these people we... -0.210614 -0.471415   
4     The Third Wave developed  rapidly because the ...  3.272894  3.219757   
...                                                 ...       ...       ...   
7160  They used all sorts of chemical concoctions to...  0.205

In [19]:
#FEATURE CREATION
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from tqdm import tqdm
tqdm.pandas()

features_df = pd.DataFrame()

features_df["num_words"] = summary_df["text"].apply(lambda x: len(word_tokenize(x)))

def count_punctuation(text):
    return sum(1 for char in text if char in string.punctuation)

features_df['punctuation_count'] = merged_df['text'].apply(count_punctuation)

def count_words(text):
    return len(text.split()) 

features_df['response_length'] = merged_df['text'].apply(count_words)

def find_similar_words(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def count_repeated_words(response, prompt):
    repeated_words = set()
    prompt_words = set(prompt.split())
    for word in response.split():
        if word in prompt_words:
            repeated_words.add(word)
        else:
            synonyms = find_similar_words(word)
            if synonyms.intersection(prompt_words):
                repeated_words.add(word)
    return len(repeated_words)

# Calculate the number of repeated words for each response text
features_df['repeated_words_prompt_text'] = merged_df.apply(lambda row: count_repeated_words(row['text'], row['prompt_text']), axis=1)

features_df['repeated_words_prompt_question'] = merged_df.apply(lambda row: count_repeated_words(row['text'], row['prompt_question']), axis=1)

from nltk.corpus import stopwords

nltk.download('stopwords')

# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Function to count the number of stopwords in a text
def count_stopwords(text):
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stop_words)
    return stopwords_count

# Calculate the number of stopwords for each response
features_df['stopwords_count'] = merged_df['text'].apply(count_stopwords)

from collections import Counter
def calculate_repetition(text):
    # Tokenize the text into words
    words = text.lower().split()
    # Count the frequency of each word
    word_counts = Counter(words)
    # Calculate repetition metric (e.g., total number of repeated words)
    repetition = sum(count for word, count in word_counts.items() if count > 1)
    return repetition

features_df['repetition'] = merged_df['text'].apply(calculate_repetition)

from nltk.tokenize import word_tokenize

mean_word_lengths = []

# Iterate over each text in the 'text' column
for text in summary_df['text']:
    # Tokenize the text into words
    words = word_tokenize(text)
    # Calculate the mean word length for the text and append it to the mean_word_lengths list
    mean_word_lengths.append(sum(len(word) for word in words) / len(words) if words else 0)

# Add the mean word lengths as a new column in the DataFrame
features_df['mean word length'] = mean_word_lengths

def is_any_alpha(input_string):
    return any(char.isalpha() for char in input_string)

def extract_proper_nouns(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    proper_nouns = set()
    for tagged_word in tagged_words:
        if isinstance(tagged_word, tuple) and tagged_word[1] == 'NNP':
            if is_any_alpha(tagged_word[0]) is True:
                proper_nouns.add(tagged_word[0])
    return list(proper_nouns)

def prop_noun_intersection(prompt_prop_nouns, summary):
    prompt_proper_nouns = set(prompt_prop_nouns)
    summary_proper_nouns = set(extract_proper_nouns(summary))
    return len(prompt_proper_nouns.intersection(summary_proper_nouns))/len(prompt_proper_nouns)

prompt_prop_nouns = {}
for i, prompt in prompts_df.iterrows():
    prompt_prop_nouns[prompt["prompt_id"]] = extract_proper_nouns(prompt["prompt_text"])

features_df["proper_noun_intersection"] = merged_df.progress_apply(lambda row: prop_noun_intersection(prompt_prop_nouns[row["prompt_id"]], row["text"]), axis=1)

def split_quotations(text: str) -> (str, List[str]):
    quotations = re.findall('"([^"]*)"', text)
    no_quote_text = text
    for quotation in quotations:
        no_quote_text = no_quote_text.replace(f'"{quotation}"', "")
    return no_quote_text, quotations

def count_quotations(text: str) -> int:
    return len(re.findall('"([^"]*)"', text))

features_df["quoteCount"] = summary_df["text"].apply(count_quotations)

def avg_quote_length(text: str) -> float:
    no_quote_text, quotations = split_quotations(text)
    if len(quotations) == 0:
        return 0
    return sum(len(q) for q in quotations) / len(quotations)

features_df["avgQuoteLength"] = summary_df["text"].apply(avg_quote_length)

from textblob import TextBlob

def get_polarity(text: str) -> float:
    blob = TextBlob(text)
    return blob.sentiment.polarity

features_df["polarity"] = summary_df["text"].progress_apply(get_polarity)

def get_subjectivity(text: str) -> float:
    blob = TextBlob(text)
    return blob.sentiment.subjectivity

features_df["subjectivity"] = summary_df["text"].progress_apply(get_subjectivity)

def textblob_pos_filter(text: str, pos_tag: str) -> List[str]:
    blob = TextBlob(text)
    return [word for word, pos in blob.tags if pos.startswith(pos_tag)]

def count_duplicates(words: List[str]) -> dict:
    word_counts = {}
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1
    return word_counts

def num_adjectives_repeated(text: str) -> int:
    adjectives = textblob_pos_filter(text, "JJ")
    return len([word for word, count in count_duplicates(adjectives).items() if count >= 2])

features_df["numAdjectivesRepeated2p"] = summary_df["text"].progress_apply(num_adjectives_repeated)

def prop_adjectives_repeated_text_normalised(text: str) -> float:
    adjectives = textblob_pos_filter(text, "JJ")
    if len(adjectives) == 0:
        return None
    return len([word for word, count in count_duplicates(adjectives).items() if count >= 2]) / len(text)

features_df["propAdjectivesRepeated2pTextNormalised"] = summary_df["text"].progress_apply(prop_adjectives_repeated_text_normalised)

import textstat

def felsch_reading_ease(text: str) -> float:
    return textstat.flesch_reading_ease(text)

features_df["fleschReadingEase"] = summary_df["text"].progress_apply(felsch_reading_ease)

features_df["num_sentences"] = summary_df["text"].apply(lambda x: len(nltk.sent_tokenize(x)))
features_df["words_per_sentence"] = features_df["num_words"] / features_df["num_sentences"]


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joepa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\joepa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joepa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\joepa\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\joepa\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-

In [33]:
#MODELS
import xgboost
model = xgboost.XGBRegressor()
content_target = summary_df['content']
content_target = pd.DataFrame(content_target, columns=['content'])
wording_target = summary_df['wording']
wording_target = pd.DataFrame(wording_target, columns=['wording'])


print(cross_validate(model, features_df, content_target))

Using 10x10 cross validation


  0%|          | 0/100 [00:00<?, ?it/s]

Metric        rmse                       mae                        r2  \
Target     content mean_columnwise   content mean_columnwise   content   
mean      0.493849        0.493849  0.370082        0.370082  0.774866   
stdev     0.017314        0.017314  0.011174        0.011174  0.018446   
n_trials       100               1       100               1       100   

Metric                    
Target   mean_columnwise  
mean            0.774866  
stdev           0.018446  
n_trials               1  


In [34]:
print(cross_validate(model, features_df, wording_target))

Using 10x10 cross validation


  0%|          | 0/100 [00:00<?, ?it/s]

Metric        rmse                       mae                        r2  \
Target     wording mean_columnwise   wording mean_columnwise   wording   
mean      0.662368        0.662368  0.505357        0.505357  0.589966   
stdev     0.022384        0.022384  0.015138        0.015138  0.024588   
n_trials       100               1       100               1       100   

Metric                    
Target   mean_columnwise  
mean            0.589966  
stdev           0.024588  
n_trials               1  
